In [54]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

In [55]:
import re

In [56]:
df = pd.read_csv('../Detail/done_thong_tin_bds_thang_6_201_400.csv')

In [57]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3931 entries, 0 to 3930
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   URL                  3931 non-null   object
 1   Người bán            3931 non-null   object
 2   Loại người bán       3931 non-null   object
 3   Số phòng ngủ         3931 non-null   object
 4   Hướng                3931 non-null   object
 5   Kiểu nhà             3931 non-null   object
 6   Giá                  3931 non-null   object
 7   Giá mỗi m2           3931 non-null   object
 8   Diện tích            3931 non-null   object
 9   Địa chỉ              3931 non-null   object
 10  Giấy tờ pháp lý      3931 non-null   object
 11  Số phòng vệ sinh     3931 non-null   object
 12  Tình trạng nội thất  3931 non-null   object
 13  Số tầng              3931 non-null   object
dtypes: object(14)
memory usage: 430.1+ KB


In [58]:
df.tail(5)

,URL,Người bán,Loại người bán,Số phòng ngủ,Hướng,Kiểu nhà,Giá,Giá mỗi m2,Diện tích,Địa chỉ,Giấy tờ pháp lý,Số phòng vệ sinh,Tình trạng nội thất,Số tầng
3926,https://www.nhatot.com/mua-ban-nha-dat-quan-ha...,Phan Hùng,Môi Giới,4 PN,Không rõ,"Nhà ngõ, hẻm","4,65 tỷ",186 triệu/m²,25 m²,"Ngõ 159 Phố Hồng Mai, Phường Quỳnh Lôi, Quận H...",Đã có sổ,Không rõ,Nội thất đầy đủ,6
3927,https://www.nhatot.com/mua-ban-dat-huyen-soc-s...,Mạnh,Cá nhân,Không rõ,Không rõ,Đất thổ cư,"2,52 tỷ","25,20 triệu/m²",100 m²,"Phú Hạ, Xã Minh Phú, Huyện Sóc Sơn, Hà Nội",Đã có sổ,Không rõ,Không rõ,Không rõ
3928,https://www.nhatot.com/mua-ban-nha-dat-quan-ho...,Huê Nguyễn,Môi Giới,4 PN,Không rõ,"Nhà ngõ, hẻm","8,65 tỷ","201,16 triệu/m²",43 m²,"Đường Kim Giang, Phường Thanh Trì, Quận Hoàng ...",Đã có sổ,Không rõ,Không rõ,4
3929,https://www.nhatot.com/mua-ban-nha-dat-quan-th...,Vũ Minh Quân,Môi Giới,4 PN,Không rõ,"Nhà ngõ, hẻm",8 tỷ,"195,12 triệu/m²",41 m²,"Phố Vương Thừa Vũ, Phường Khương Trung, Quận T...",Đã có sổ,3 phòng,Nội thất đầy đủ,4
3930,https://www.nhatot.com/mua-ban-nha-dat-quan-do...,Tên chưa cung cấp,Môi Giới,4 PN,Không rõ,"Nhà ngõ, hẻm","13,6 tỷ","323,81 triệu/m²",42 m²,"Phố Tây Sơn, Phường Quang Trung, Quận Đống Đa,...",Đã có sổ,5 phòng,Không rõ,Không rõ


In [59]:
df['Số phòng ngủ'].unique()

array(['4 PN', '1 PN', '3 PN', '5 PN', '2 PN', 'Không rõ', '7 PN',
       'nhiều hơn 10 PN', '6 PN', '9 PN', '8 PN', '10 PN',
       '4 PN • Nhà ngõ, hẻm'], dtype=object)

In [60]:
df['Số phòng ngủ'] = df['Số phòng ngủ'].str.replace('nhiều hơn 10', '10+').str.strip()

In [61]:
df['Số phòng ngủ'] = df['Số phòng ngủ'].str.replace('PN', ' ').str.strip()

In [62]:
df['Số phòng vệ sinh'].unique()

array(['Không rõ', '1 phòng', '4 phòng', '2 phòng', 'Nhiều hơn 6 phòng',
       '3 phòng', '5 phòng', '6 phòng', '3', '5', '4', '6+', '1', '2'],
      dtype=object)

In [63]:
df['Số phòng vệ sinh'] = df['Số phòng vệ sinh'].str.replace('Nhiều hơn 6', '6+').str.strip()
df['Số phòng vệ sinh'] = df['Số phòng vệ sinh'].str.replace('phòng', ' ').str.strip()

In [64]:
def convert_price(value):
    if pd.isna(value):
        return None
    value = str(value).strip()

    value = value.replace(",", ".")

    if "tỷ" in value:
        num = float(value.replace("tỷ/m²", "").replace("tỷ", "").strip())
        return num * 1000  # đổi 1 tỷ -> 1000 triệu
    elif "triệu" in value:
        num = float(value.replace("triệu/m²", "").replace("triệu", "").strip())
        return num
    elif "đ/m²" in value:
        num = float(value.replace("đ/m²", "").replace("đ", "").strip())
        return num / 1000
    elif "Khôngrõ" in value:
        return 0
    elif "Không rõ" in value:
        return 0
    else:
        return float(value)  # nếu không rõ đơn vị, giả định là triệu


In [65]:
df['Giá mỗi m2'] = df['Giá mỗi m2'].apply(convert_price)

In [66]:
df['Giá'] = df['Giá'].apply(convert_price)

In [67]:
df['Diện tích'] = df['Diện tích'].str.replace('m²', '').str.replace(',', '.').str.replace('Không rõ', '0').astype(float)

In [68]:
df['Số tầng'] = df['Số tầng'].str.replace('Không rõ', '0').astype(int)

In [69]:
def split_address(address):
    result = {
        'Đường/Phố': "Không rõ",
        'Phường/Xã/Thị trấn': "Không rõ",
        'Quận/Huyện': "Không rõ",
        'Thành phố': "Không rõ"
    }

    parts = [p.strip() for p in address.split(',')]
    
    for p in parts:
        if re.search(r'Phường|Xã|Thị trấn', p):
            result['Phường/Xã/Thị trấn'] = p
        elif re.search(r'Quận|Huyện', p):
            result['Quận/Huyện'] = p
        elif re.search(r'Hà Nội|Hồ Chí Minh|TP', p):
            result['Thành phố'] = p
        else:
            result['Đường/Phố'] = p

    return pd.Series(result)

# Giả sử df là DataFrame của bạn
df_split = df['Địa chỉ'].apply(split_address)
df = df.join(df_split)

In [70]:
df = df.drop(columns=['Địa chỉ'])

EDA DỮ LIỆU

In [71]:
df[df.duplicated()]

,URL,Người bán,Loại người bán,Số phòng ngủ,Hướng,Kiểu nhà,Giá,Giá mỗi m2,Diện tích,Giấy tờ pháp lý,Số phòng vệ sinh,Tình trạng nội thất,Số tầng,Đường/Phố,Phường/Xã/Thị trấn,Quận/Huyện,Thành phố


In [72]:
df = df[(df['Giá'] != 0) & (df['Diện tích'] != 0)]

In [74]:
df.to_csv('../Data_Cleaned/thong_tin_bds_thang_6_201_400_cleaned.csv', index=False, encoding='utf-8-sig')

print("Dữ liệu đã được nối và lưu vào file thong_tin_bds_thang_6_201_300_cleaned.csv")

Dữ liệu đã được nối và lưu vào file thong_tin_bds_thang_6_201_300_cleaned.csv
